# IMDB 영화평 감성분석(이진분류)

In [1]:
import numpy as np
import pandas as pd
from google.colab import files
up=files.upload()

Saving labeledTrainData.tsv to labeledTrainData.tsv


In [2]:
df=pd.read_csv('labeledTrainData.tsv',sep='\t',quoting=3) #3:quote_none. 따옴표 같은 거 제대로 들어가게.
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [3]:
df.review[0][:1000]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

- 텍스트 전처리

In [4]:
#<br />태그는 공백으로 바꾸기.
df.review = df.review.str.replace('<br />',' ')

In [5]:
#구둣점, 숫자 제거 - 영문자가 아닌 글자는 공백으로.
#데이터 프레임의 str method는 정규표현식을 지원. 
#근데 그냥 str 형식에서는 이거 못씀.
df.review = df.review.str.replace('[^A-Za-z]',' ').str.strip()
df.review[0][:1000]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


'With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay   Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him   The actual feature film bit when it finally starts is only on for  

- train/test 분리

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(
    df.review, df.sentiment, stratify=df.sentiment, random_state=2022
)
y_train.value_counts()

0    9375
1    9375
Name: sentiment, dtype: int64

- CountVectorizer로 변환

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
cvect=CountVectorizer(stop_words='english')

In [20]:
cvect.fit(X_train)
X_train_cv=cvect.transform(X_train)
X_train_cv.shape

(18750, 65213)

In [21]:
#test dataset은 train dataset을 변환한 vectorizer로 변환해야 함.
X_test_cv=cvect.transform(X_test)
X_test_cv.shape

(6250, 65213)

- 분류기:SVM

- 분류기:Naive Bayes

In [23]:
from sklearn.naive_bayes import MultinomialNB
nb=MultinomialNB()


In [24]:
%time nb.fit(X_train_cv, y_train)

CPU times: user 21 ms, sys: 795 µs, total: 21.8 ms
Wall time: 21.2 ms


MultinomialNB()

In [25]:
nb.score(X_test_cv,y_test)

0.85504

In [17]:
cvect2=CountVectorizer(ngram_range=(1,2),stop_words='english')
cvect2.fit(X_train)
%time X_train_cv2=cvect2.transform(X_train)
print(X_train_cv2.shape)

CPU times: user 4.53 s, sys: 5.32 ms, total: 4.53 s
Wall time: 4.51 s
(18750, 1386558)


In [28]:
X_test_cv2=cvect2.transform(X_test)
X_test_cv2.shape

(6250, 1386558)

In [29]:
nb2=MultinomialNB()
%time nb2.fit(X_train_cv2, y_train)

CPU times: user 135 ms, sys: 3.65 ms, total: 139 ms
Wall time: 137 ms


MultinomialNB()

In [30]:
nb2.score(X_test_cv2,y_test)

0.8672

- 모델 저장하고 불러오기

In [31]:
import joblib
joblib.dump(cvect2, 'imdb_cvect12.pkl')
joblib.dump(nb2,'imdb_nb2.pkl')

['imdb_nb2.pkl']

In [32]:
!ls -l

total 114376
-rw-r--r-- 1 root root 39184702 Apr 28 02:39 imdb_cvect12.pkl
-rw-r--r-- 1 root root 44370573 Apr 28 02:39 imdb_nb2.pkl
-rw-r--r-- 1 root root 33556378 Apr 28 02:28 labeledTrainData.tsv
drwxr-xr-x 1 root root     4096 Apr 25 13:46 sample_data


In [33]:
new_cvect=joblib.load('imdb_cvect12.pkl')
new_nb=joblib.load('imdb_nb2.pkl')

- 실제 데이터로 검증

In [35]:
review='''
I'll admit I raised an eyebrow when I saw that Pattinson was cast, but I eat my words, he was awesome, and hopefully will play the part a few more times.
This film blew me away, exciting, fast paced, surprisingly gritty, and genuinely had an awesome story.
The franchise somehow felt renewed here, even if you're not a fan of superhero movies, you will love this.
Andy Serkis was genuinely awesome, but when isn't he? The acting throughout was fantastic, with plenty of well known faces.
Great music, great special effects, I've already booked to see it again.
'''


In [37]:
#텍스트 전처리
import re
review = re.sub('[^A-Za-z]',' ',review).strip()

In [39]:
#feature 변환
review_cv=new_cvect.transform([review])

In [41]:
new_nb.predict(review_cv)

array([1])